In [2]:
import os
import json
from PIL import Image
import json

with open("/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train/_annotations.coco.json", "r") as f:
    data = json.load(f)
print("file COCO:", list(data.keys()))

# Đường dẫn
json_path = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train/_annotations.coco.json"
image_dir = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train"
cropped_dir = "/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped"

# Tạo thư mục lưu ảnh cắt
os.makedirs(cropped_dir, exist_ok=True)

# Tạo dict ánh xạ từ image_id => image_info ( thông tin ảnh )
image_dict = {img_info["id"]: img_info for img_info in data["images"]}

# Tạo dict ánh xạ category_id -> category_name
category_dict = {cat["id"]: cat["name"] for cat in data["categories"]}

file COCO: ['info', 'licenses', 'categories', 'images', 'annotations']


In [3]:
# read Fieldplant

deleted_class = ['Tomato leaf mosaic virus', 'Tomato bacterial wilt']
class_names = [cls for cls in os.listdir(cropped_dir) 
                if (os.path.isdir(os.path.join(cropped_dir, cls)) 
                & ('tomato' in cls.lower()) 
                & (cls not in deleted_class)
                )]
class_names

image_paths = []
labels = []
for cls in class_names:
    class_folder = os.path.join(cropped_dir, cls)
    for fname in os.listdir(class_folder):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_paths.append(os.path.join(class_folder, fname))
            labels.append(cls)

%store image_paths labels

len(image_paths), len(labels)

Stored 'image_paths' (list)
Stored 'labels' (list)


(2765, 2765)

In [4]:
from collections import Counter
list_labels = Counter(labels)

label2id = {label: id for id, label in enumerate(list_labels)}
id2label = {id: label for id, label in enumerate(list_labels)}
labels = [label2id[label] for label in labels]
list_labels, labels[:3]

(Counter({'Tomato Brown Spots': 1967,
          'Tomato blight leaf': 404,
          'Tomato healthy': 279,
          'Tomato leaf yellow virus': 115}),
 [0, 0, 0])

In [5]:
import os
import shutil

def move_except_keep(old_dir, new_dir, keep_list):
    os.makedirs(new_dir, exist_ok=True)
    
    for item in os.listdir(old_dir):
        if item in keep_list:
            continue

        src_path = os.path.join(old_dir, item)
        dst_path = os.path.join(new_dir, item)

        shutil.move(src_path, dst_path)
        print(f"Moved: {src_path} -> {dst_path}")

# Example usage:
old_dir = '/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped'
new_dir = '/media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use'
keep_list = ['Tomato Brown Spots', 'Tomato blight leaf', 'Tomato healthy', 'Tomato leaf yellow virus']  

move_except_keep(old_dir, new_dir, keep_list)


Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Corn Gray leaf spot -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Corn Gray leaf spot
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Corn Smut -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Corn Smut
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Corn Brown Spots -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Corn Brown Spots
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomato leaf mosaic virus -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Tomato leaf mosaic virus
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Tomato bacterial wilt -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Tomato bacterial wilt
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Cassava Bacterial Blight -> /media/icnlab/Data/Manh/tinyML/FieldPlant-11/no_use/Cassava Bacterial Blight
Moved: /media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped/Corn rust le

In [27]:
import pandas as pd
df = pd.DataFrame({
    'label': labels,
    'file_path': image_paths,
})
df.to_csv('/media/icnlab/Data/Manh/tinyML/FieldPlant-11/data.csv', index=True)

In [29]:
from sklearn.model_selection import train_test_split
# Chia 80% train, 20% test
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths,
    labels,
    test_size=0.2,
    stratify=labels,     # giữ phân bố nhãn nếu cần
    random_state=42      # để kết quả có thể tái lặp
)

# Kiểm tra kết quả
print(f"Số ảnh train: {len(train_paths)}")
print(f"Số ảnh test: {len(test_paths)}")

Số ảnh train: 2212
Số ảnh test: 553


In [31]:
import pandas as pd
df = pd.DataFrame({
    'label': train_labels,
    'file_path': train_paths,
})
df.to_csv('/media/icnlab/Data/Manh/tinyML/FieldPlant-11/train.csv', index=True)

import pandas as pd
df = pd.DataFrame({
    'label': test_labels,
    'file_path': test_paths,
})
df.to_csv('/media/icnlab/Data/Manh/tinyML/FieldPlant-11/test.csv', index=True)

In [8]:
# move images to class folders
import os
import shutil
def move_images_to_folder(source_folder):
    for f in os.listdir(source_folder):
        if f.endswith('.jpg'):
            label = f.split('_')[-1].split('.')[0]
            destination_folder = os.path.join(source_folder, label)
            if not os.path.exists(destination_folder):
                os.makedirs(destination_folder)
            source_path = os.path.join(source_folder, f)
            destination_path = os.path.join(destination_folder, f)
            shutil.move(source_path, destination_path)

source_folder = '/media/icnlab/Data/Manh/tinyML/FieldPlant-11/cropped'
move_images_to_folder(source_folder)

In [ ]:
# Cắt ảnh
for ann in data["annotations"]:
    image_id = ann["image_id"]
    img_info = image_dict[image_id]

    category_id = ann["category_id"]
    category_name = category_dict[category_id]

    bbox = ann["bbox"]  # [x, y, width, height]
    img_path = os.path.join(image_dir, img_info["file_name"])
    
    # Mở ảnh
    try:
        with Image.open(img_path) as im:
            x, y, w, h = map(int, bbox)
            cropped = im.crop((x, y, x + w, y + h))

            # Tạo tên file output
            base_name = os.path.splitext(img_info["file_name"])[0]
            out_name = f"{base_name}_{ann['id']}_{category_name}.jpg"
            out_path = os.path.join(output_dir, out_name)

            # Lưu ảnh cắt
            cropped.save(out_path)
    except Exception as e:
        print(f"Lỗi với ảnh {img_path}: {e}")